In [1]:
pip install python-Levenshtein

In [47]:
import pandas as pd
from Levenshtein import distance
import re

class SinhalaLanguageChecker:
    def __init__(self, spell_dict_path, grammar_dict_path):
        """
        Initialize the language checker with dictionaries for spelling and grammar
        """
        try:
            # Load spelling dictionary
            self.spell_df = pd.read_excel(spell_dict_path)
            # Create set of correct words for faster lookup
            self.correct_words = set(self.spell_df[self.spell_df['label'] == 1]['word'].values)

            # Load grammar rules and clean column names
            self.grammar_df = pd.read_excel(grammar_dict_path)
            self.grammar_df.columns = self.grammar_df.columns.str.strip()

            # Validate required columns exist
            required_columns = {'verb', 'subject', 'corrected_verb'}
            if not all(col in self.grammar_df.columns for col in required_columns):
                missing_cols = required_columns - set(self.grammar_df.columns)
                raise ValueError(f"Missing required columns in grammar file: {missing_cols}")

            # Create verb conjugation patterns
            self.verb_patterns = self._create_verb_patterns()

        except FileNotFoundError as e:
            raise FileNotFoundError(f"Could not find the file: {e.filename}")
        except Exception as e:
            raise Exception(f"Error initializing the checker: {str(e)}")

    def _create_verb_patterns(self):
        """Create verb conjugation patterns from the grammar dataset"""
        patterns = {}
        for _, row in self.grammar_df.iterrows():
            verb = row['verb'].strip()
            subject = row['subject'].strip()
            corrected = row['corrected_verb'].strip()

            if verb not in patterns:
                patterns[verb] = {}

            patterns[verb][subject] = corrected
        return patterns

    def tokenize(self, text):
        """Split Sinhala text into words"""
        words = re.findall(r'[\u0D80-\u0DFF]+', text)
        return [word.strip() for word in words if word.strip()]

    def get_spelling_suggestions(self, word, max_distance=2):
        """Get spelling suggestions for a word"""
        word = word.strip()
        suggestions = []

        # Find similar words using Levenshtein distance
        for correct_word in self.correct_words:
            dist = distance(word, correct_word)
            if dist <= max_distance:
                suggestions.append((correct_word, dist))

        # Return top 5 suggestions sorted by distance
        return [word for word, _ in sorted(suggestions, key=lambda x: x[1])][:5]

    def check_spelling(self, word):
        """Check if a word is spelled correctly and get suggestions"""
        word = word.strip()
        if word in self.correct_words:
            return True, word, []
        else:
            suggestions = self.get_spelling_suggestions(word)
            return False, suggestions[0] if suggestions else word, suggestions

    def conjugate_verb(self, verb, subject):
        """Conjugate verb based on subject"""
        verb = verb.strip()
        subject = subject.strip()

        if verb in self.verb_patterns and subject in self.verb_patterns[verb]:
            return self.verb_patterns[verb][subject]
        return verb

    def correct_sentence(self, sentence):
        """
        Correct both spelling and grammar while preserving content.
        Detect subject dynamically in the sentence and adjust verb conjugation.
        """
        words = self.tokenize(sentence)
        if len(words) < 2:
            return sentence, False, {}
    
        # Initialize variables
        spelling_errors = {}
        corrected_words = []
        is_changed = False
        subject = None
    
        # Identify the subject from the sentence
        for word in words:
            if word in self.grammar_df['subject'].values:
                subject = word
                break
    
        if not subject:
            # If no subject is found, return as is
            return sentence, False, {}
    
        # Check and correct spelling for all words
        for word in words:
            is_correct, corrected_word, suggestions = self.check_spelling(word)
            if not is_correct:
                spelling_errors[word] = suggestions
                corrected_words.append(corrected_word)
                is_changed = True
            else:
                corrected_words.append(word)
    
        # Correct verb based on the identified subject
        for idx, word in enumerate(corrected_words):
            if word in self.verb_patterns:
                corrected_verb = self.conjugate_verb(word, subject)
                if corrected_verb != word:
                    corrected_words[idx] = corrected_verb
                    is_changed = True
    
        corrected_sentence = ' '.join(corrected_words)
        return corrected_sentence, is_changed, spelling_errors


    def check_text(self, text):
        """
        Check and correct text while preserving content
        """
        result = {
            'spelling_errors': {},
            'grammar_analysis': []
        }

        # Split into sentences
        sentences = re.split(r'[။।.]', text)
        sentences = [s.strip() for s in sentences if s.strip()]

        for sentence in sentences:
            corrected, is_changed, spelling_errors = self.correct_sentence(sentence)
            result['spelling_errors'].update(spelling_errors)
            result['grammar_analysis'].append({
                'sentence': sentence,
                'is_correct': not is_changed,
                'suggested_correction': corrected if is_changed else sentence,
                'confidence': 1.0 if not is_changed else 0.8
            })

        return result

def main():
    try:
        checker = SinhalaLanguageChecker(
            'data-spell-checker.xlsx',
            'grammar.xlsx'
        )

        # Test texts
        test_texts = [
            "අපි ගිම්හානයේදී වෙරළට යාමට කැමතියි. වැල්ලේ වොලිබෝල් ක්‍රීඩා කිරීම සහ මුහුදේ පිහිනීම අත්දැකීම ප්‍රීතිමත් කරයි. ඉන්පසුව, අපි සාමාන්‍යයෙන් ගිනි මැලයක් වටා රැස් වී තරු යට කථා කරමින් සිටියා.",
            "මම සිංහල සිංදු ඇසීමට ආස කරනවා",
            "මම ජගම දුරකතණය භාවිත කරනවා"
        ]

        for text in test_texts:
            print(f"\nChecking text: {text}")
            results = checker.check_text(text)

            print("\nGrammar Analysis:")
            for analysis in results['grammar_analysis']:
                print(f"Original: {analysis['sentence']}")
                if not analysis['is_correct']:
                    print(f"Suggestion: {analysis['suggested_correction']}")

            if results['spelling_errors']:
                print("\nSpelling Errors:")
                for error, suggestions in results['spelling_errors'].items():
                    print(f"Error: {error}")
                    print(f"Suggestions: {suggestions}")

    except Exception as e:
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    main()


Checking text: අපි ගිම්හානයේදී වෙරළට යාමට කැමතියි. වැල්ලේ වොලිබෝල් ක්‍රීඩා කිරීම සහ මුහුදේ පිහිනීම අත්දැකීම ප්‍රීතිමත් කරයි. ඉන්පසුව, අපි සාමාන්‍යයෙන් ගිනි මැලයක් වටා රැස් වී තරු යට කථා කරමින් සිටියා.

Grammar Analysis:
Original: අපි ගිම්හානයේදී වෙරළට යාමට කැමතියි
Suggestion: අපි ගිම්හානයේදී වෙරළට යාම කැමිය
Original: වැල්ලේ වොලිබෝල් ක්‍රීඩා කිරීම සහ මුහුදේ පිහිනීම අත්දැකීම ප්‍රීතිමත් කරයි
Original: ඉන්පසුව, අපි සාමාන්‍යයෙන් ගිනි මැලයක් වටා රැස් වී තරු යට කථා කරමින් සිටියා
Suggestion: ඉන්පසු අපි සාමානය හයෙන් පිනි සැටයක් වටා රැස් වී තුරු යට කථා හරමින් සිටියා

Spelling Errors:
Error: ගිම්හානයේදී
Suggestions: []
Error: යාමට
Suggestions: ['යාම', 'යාමක', 'සුමට', 'චාම්', 'හමට']
Error: කැමතියි
Suggestions: ['කැමිය', 'කැමති', 'කැමතිම']
Error: ඉන්පසුව
Suggestions: ['ඉන්පසු', 'ඉස්පාසුව']
Error: සාමාන්
Suggestions: ['සාමානය', 'සාමන්', 'සාමීන්', 'සාමින්', 'සාමාන']
Error: යයෙන්
Suggestions: ['හයෙන්', 'යෙන්', 'සයෙන්', 'සෑයෙන්', 'හයින්']
Error: ගිනි
Suggestions: ['පිනි', 'සිනි', 'ගිහි', 'සිසි', 'ගෝනා

In [23]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [5]:
subject_verb_endings = {
    "මම": {"present": "න්නෙමි"},
    "ඔයා": {"present": "න්නෙහ්"}
}

def correct_sentence(sentence):
    words = sentence.split()
    subject = words[0]  # Assuming the subject is the first word
    verb_index = 2 # Assuming verb is the third word
    verb = words[verb_index]

    if subject in subject_verb_endings:
        correct_ending = subject_verb_endings[subject]["present"]
        corrected_verb = verb[:-2] + correct_ending  # Simple replacement (needs improvement)
        words[verb_index] = corrected_verb
        return " ".join(words)
    return sentence # If subject not found no correction

incorrect_sentence = "මම යනවා"
corrected_sentence = correct_sentence(incorrect_sentence)
print(f"Incorrect: {incorrect_sentence}")
print(f"Corrected: {corrected_sentence}")

incorrect_sentence = "ඔයා යනවා"
corrected_sentence = correct_sentence(incorrect_sentence)
print(f"Incorrect: {incorrect_sentence}")
print(f"Corrected: {corrected_sentence}")

IndexError: list index out of range

In [6]:
subject_verb_endings = {
    "මම": {"present": "න්නෙමි"},
    "ඔයා": {"present": "න්නෙහ්"}
}

def correct_sentence(sentence):
    words = sentence.split()
    if not words:  # Handle empty sentences
        return ""

    subject = words[0]
    verb_index = -1

    # Find the verb (Improved verb identification - basic)
    for i, word in enumerate(words[1:]): # skip the first word which is subject
        if word.endswith("වා") or word.endswith("නවා"): # Basic check for present tense verb ending
            verb_index = i+1 # since we skip the first word we add 1 to the index
            break
    if verb_index == -1:
        return sentence

    verb = words[verb_index]

    if subject in subject_verb_endings:
        if "present" in subject_verb_endings[subject]:
            correct_ending = subject_verb_endings[subject]["present"]
            if verb.endswith("වා"):
                corrected_verb = verb[:-2] + correct_ending
            elif verb.endswith("නවා"):
                corrected_verb = verb[:-4] + correct_ending
            else:
                corrected_verb = verb + correct_ending # If no known suffix just add ending
            words[verb_index] = corrected_verb
            return " ".join(words)
        else:
            return sentence
    return sentence

# Test cases
test_cases = [
    "මම යනවා",
    "ඔයා යනවා",
    "මම කනවා",
    "ඔයා කනවා",
    "මම ගමන් කරනවා",
    "This is a test", # No Sinhala verb
    "", #Empty String
    "මම" #Only Subject
]

for incorrect_sentence in test_cases:
    corrected_sentence = correct_sentence(incorrect_sentence)
    print(f"Incorrect: '{incorrect_sentence}'")
    print(f"Corrected: '{corrected_sentence}'")
    print("-" * 20)

Incorrect: 'මම යනවා'
Corrected: 'මම යනන්නෙමි'
--------------------
Incorrect: 'ඔයා යනවා'
Corrected: 'ඔයා යනන්නෙහ්'
--------------------
Incorrect: 'මම කනවා'
Corrected: 'මම කනන්නෙමි'
--------------------
Incorrect: 'ඔයා කනවා'
Corrected: 'ඔයා කනන්නෙහ්'
--------------------
Incorrect: 'මම ගමන් කරනවා'
Corrected: 'මම ගමන් කරනන්නෙමි'
--------------------
Incorrect: 'This is a test'
Corrected: 'This is a test'
--------------------
Incorrect: ''
Corrected: ''
--------------------
Incorrect: 'මම'
Corrected: 'මම'
--------------------


In [12]:
formal_sinhala_conjugations = {
    "present": {
        "singular": {
            "1st person": {"මම": ["න්නෙමි", "මි"]},  # කරන්නෙමි/කරමි
            "2nd person": {"ඔබ": ["න්නෙහි", "හි"]},  # කරන්නෙහි/කරහි
            "3rd person masculine": {"ඔහු": ["යි"]},      # කරයි
            "3rd person feminine": {"ඇය": ["යි"]},      # කරයි
            "3rd person neuter": {"ඒ/එය": ["යි"]},      # කරයි
        },
        "plural": {
            "1st person": {"අපි": ["න්නෙමු", "මු"]},  # කරන්නෙමු/කරමු
            "2nd person": {"ඔබලා/නුඹලා": ["න්නෙහු", "හු"]},# කරන්නෙහු/කරහු
            "3rd person": {"ඔවුන්": ["ති"]},      # කරති
            "3rd person neuter": {"ඒවා": ["ති"]},      # කරති
        }
    },
    "past": {
        "singular": {
            "1st person": {"මම": ["-ීමි", "-ුවෙමි", "-ුයෙමි","ඇමි","අස්මි"]},  # කෙරුවෙමි/කෙරුයෙමි/කළෙමි/කළාමි/කළස්මි
            "2nd person": {"ඔබ": ["-ුවෙහි", "-ුව", "-ුයෙහි", "-ඇයෙහි", "අසෙහි"]},  # කෙරුවෙහි/කෙරුව/කෙරුයෙහි/කළෙයෙහි/කළසෙහි
            "3rd person masculine": {"ඔහු": ["-ුවේය", "-ුවා", "-ුයේය","ඇයේය","ඇස්සය"]},  # කෙරුවේය/කෙරුවා/කෙරුයේය/කළේය/කළස්සය
            "3rd person feminine": {"ඇය": ["-ුවාය", "-ුවා", "-ුයාය", "ඇයාය","ඇස්සාය"]},  # කෙරුවාය/කෙරුවා/කෙරුයාය/කළාය/කළස්සාය
            "3rd person neuter": {"ඒ/එය": ["-ුවේය", "-ුවා", "-ුයේය","ඇයේය","ඇස්සය"]},  # කෙරුවේය/කෙරුවා/කෙරුයේය/කළේය/කළස්සය
        },
        "plural": {
            "1st person": {"අපි": ["-ුවෙමු", "-ුව", "-ුයෙමු","ඇවමු","ඇස්සුමු"]}, # කෙරුවෙමු/කෙරුව/කෙරුයෙමු/කළෙමු/කළස්සුමු
            "2nd person": {"ඔබලා/නුඹලා": ["-ුවෙහු", "-ුව", "-ුයෙහු","ඇවහු","ඇස්සුහු"]},# කෙරුවෙහු/කෙරුව/කෙරුයෙහු/කළෙහු/කළස්සුහු
            "3rd person": {"ඔවුන්": ["-ුවෝය", "-ුවා", "-ුයෝය","ඇවෝය","ඇස්සෝය"]}, # කෙරුවෝය/කෙරුවා/කෙරුයෝය/කළෝය/කළස්සෝය
            "3rd person neuter": {"ඒවා": ["-ුවෝය", "-ුවා", "-ුයෝය","ඇවෝය","ඇස්සෝය"]}, # කෙරුවෝය/කෙරුවා/කෙරුයෝය/කළෝය/කළස්සෝය
        }
    },
    "future":{
        "singular": {
            "1st person": {"මම": ["න්නම්", "මි"]},
            "2nd person": {"ඔබ": ["න්නෙහි", "හි"]},
            "3rd person masculine": {"ඔහු": ["යි", "න්නේය"]},
            "3rd person feminine": {"ඇය": ["යි", "න්නේය"]},
            "3rd person neuter": {"ඒ/එය": ["යි", "න්නේය"]},
        },
        "plural": {
            "1st person": {"අපි": ["න්නෙමු", "මු", "න්නෙම්"]},
            "2nd person": {"ඔබලා/නුඹලා": ["න්නෙහු", "හු"]},
            "3rd person": {"ඔවුන්": ["ති", "න්නාහ"]},
            "3rd person neuter": {"ඒවා": ["ති", "න්නාහ"]},
        }
    }
}

In [19]:
def extract_verb_stem(verb):
    suffixes = ["නවා", "න", "ඉම", "න්න", "න්නට", "න්නෙ", "න්නෙමි","න්නෙහ්","න්නෙමු","න්නෙහු","ති","යි","වා","න්නම්","න්නේය","න්නාහ","ීමි", "-ුවෙමි", "-ුයෙමි","-ඇමි","-අස්මි","-ුවෙහි", "-ුව", "-ුයෙහි", "-ඇයෙහි", "-අසෙහි","-ුවේය", "-ුවා", "-ුයේය","-ඇයේය","-ඇස්සය","-ුවාය", "-ුවා", "-ුයාය", "-ඇයාය","-ඇස්සාය","-ුවෙමු", "-ුව", "-ුයෙමු","-ඇවමු","-ඇස්සුමු","-ුවෙහු", "-ුව", "-ුයෙහු","-ඇවහු","-ඇස්සුහු","-ුවෝය", "-ුවා", "-ුයෝය","-ඇවෝය","-ඇස්සෝය"]
    for suffix in suffixes:
        if verb.endswith(suffix):
            return verb[:-len(suffix)]
    return verb
    
def correct_sentence(sentence):
    words = sentence.split()
    if not words:
        return ""

    subject = words[0]
    verb_index = -1

    for i, word in enumerate(words[1:]):
        if any(word.endswith(suffix) for suffix in ["වා", "නවා","යි","ති","න්නම්","න්නේය","න්නාහ","ීමි", "-ුවෙමි", "-ුයෙමි","-ඇමි","-අස්මි","-ුවෙහි", "-ුව", "-ුයෙහි", "-ඇයෙහි", "-අසෙහි","-ුවේය", "-ුවා", "-ුයේය","-ඇයේය","-ඇස්සය","-ුවාය", "-ුවා", "-ුයාය", "-ඇයාය","-ඇස්සාය","-ුවෙමු", "-ුව", "-ුයෙමු","-ඇවමු","-ඇස්සුමු","-ුවෙහු", "-ුව", "-ුයෙහු","-ඇවහු","-ඇස්සුහු","-ුවෝය", "-ුවා", "-ුයෝය","-ඇවෝය","-ඇස්සෝය"]):
            verb_index = i + 1
            break

    if verb_index == -1:
        return sentence

    verb = words[verb_index]
    verb_stem = extract_verb_stem(verb)
    tense = "present"  # Default tense (still needs improvement)

    try:
        endings = formal_sinhala_conjugations[tense]["singular" if subject in ["මම", "ඔබ", "ඔහු", "ඇය", "ඒ/එය"] else "plural"][
            "1st person" if subject in ["මම", "අපි"] else
            "2nd person" if subject in ["ඔබ", "ඔබලා/නුඹලා"] else
            "3rd person"
        ][subject]

        best_ending = None
        for ending in endings:
            if verb.endswith(ending[-len(extract_verb_stem(verb)):]) or not any(verb.endswith(suffix) for suffix in ["වා", "නවා","යි","ති","න්නම්","න්නේය","න්නාහ","ීමි", "-ුවෙමි", "-ුයෙමි","-ඇමි","-අස්මි","-ුවෙහි", "-ුව", "-ුයෙහි", "-ඇයෙහි", "-අසෙහි","-ුවේය", "-ුවා", "-ුයේය","-ඇයේය","-ඇස්සය","-ුවාය", "-ුවා", "-ුයාය", "-ඇයාය","-ඇස්සාය","-ුවෙමු", "-ුව", "-ුයෙමු","-ඇවමු","-ඇස්සුමු","-ුවෙහු", "-ුව", "-ුයෙහු","-ඇවහු","-ඇස්සුහු","-ුවෝය", "-ුවා", "-ුයෝය","-ඇවෝය","-ඇස්සෝය"]):
                best_ending = ending
                break
        if best_ending is None:
            best_ending = endings[0]

        corrected_verb = verb_stem + best_ending
        words[verb_index] = corrected_verb
        return " ".join(words)

    except KeyError:
        return sentence

test_cases = [
    ("මම යනවා", "මම යන්නෙමි"),
    ("ඔයා යනවා", "ඔයා යන්නෙහි"),  # Corrected expected output
    ("මම කනවා", "මම කන්නෙමි"),
    ("ඔයා කන්නවා", "ඔයා කන්නෙහි"),  # Corrected expected output
    ("මම ගමන් කරනවා", "මම ගමන් කරන්නෙමි"),
    ("ඔහු කරයි", "ඔහු කරයි"),
    ("ඔවුන් කරති", "ඔවුන් කරති"),
    ("මම යමි", "මම යන්නෙමි"),
    ("අපි කරමු", "අපි කරන්නෙමු"),
    ("This is a test", "This is a test"),
    ("", ""),
    ("මම", "මම"),
    ("ඔබ කනවා", "ඔබ කන්නෙහි"),
    ("මම කෙරුවා", "මම කෙරුවා"), # Past tense (not handled yet)
    ("ඔබ කෙරුවා", "ඔබ කෙරුවා"), # Past tense (not handled yet)
    ("මම කරමි","මම කරන්නෙමි"),
    ("මම කරන්නම්","මම කරන්නම්"), #future tense
    ("ඔබ කරන්නෙහි","ඔබ කරන්නෙහි"),#future tense
    ("ඔහු කරන්නේය","ඔහු කරන්නේය"),#future tense
    ("අපි කරන්නෙම","අපි කරන්නෙමු"),#future tense
    ("ඔවුන් කරන්නාහ","ඔවුන් කරන්නාහ"),#future tense
    ("මම කෑවා","මම කෑවා"),#past tense
    ("ඔබ කෑවා","ඔබ කෑවා"),#past tense
    ("ඔහු කෑවා","ඔහු කෑවා"),#past tense
    ("අපි කෑවා","අපි කෑවා"),#past tense
    ("ඔවුන් කෑවා","ඔවුන් කෑවා"),#past tense
    ("මම ගියා","මම ගියා"),#past tense
    ("ඔබ ගියා","ඔබ ගියා"),#past tense
    ("ඔහු ගියා","ඔහු ගියා"),#past tense
    ("අපි ගියා","අපි ගියා"),#past tense
    ("ඔවුන් ගියා","ඔවුන් ගියා"),#past tense
    ("මම ඉන්නවා","මම ඉන්නෙමි"),
    ("ඔබ ඉන්නවා","ඔබ ඉන්නෙහි"),
    ("ඔහු ඉන්නවා","ඔහු ඉන්නවා"),
    ("අපි ඉන්නවා","අපි ඉන්නෙමු"),
    ("ඔවුන් ඉන්නවා","ඔවුන් ඉන්නවා")
]

for incorrect_sentence, correct_sentence_expected in test_cases:
    corrected_sentence = correct_sentence(incorrect_sentence)
    print(f"Incorrect: '{incorrect_sentence}'")
    print(f"Corrected: '{corrected_sentence}'")
    print(f"Expected: '{correct_sentence_expected}'")
    print("-" * 20)


Incorrect: 'මම යනවා'
Corrected: 'මම යන්නෙමි'
Expected: 'මම යන්නෙමි'
--------------------
Incorrect: 'ඔයා යනවා'
Corrected: 'ඔයා යනවා'
Expected: 'ඔයා යන්නෙහි'
--------------------
Incorrect: 'මම කනවා'
Corrected: 'මම කන්නෙමි'
Expected: 'මම කන්නෙමි'
--------------------
Incorrect: 'ඔයා කන්නවා'
Corrected: 'ඔයා කන්නවා'
Expected: 'ඔයා කන්නෙහි'
--------------------
Incorrect: 'මම ගමන් කරනවා'
Corrected: 'මම ගමන් කරන්නෙමි'
Expected: 'මම ගමන් කරන්නෙමි'
--------------------
Incorrect: 'ඔහු කරයි'
Corrected: 'ඔහු කරයි'
Expected: 'ඔහු කරයි'
--------------------
Incorrect: 'ඔවුන් කරති'
Corrected: 'ඔවුන් කරති'
Expected: 'ඔවුන් කරති'
--------------------
Incorrect: 'මම යමි'
Corrected: 'මම යමි'
Expected: 'මම යන්නෙමි'
--------------------
Incorrect: 'අපි කරමු'
Corrected: 'අපි කරමු'
Expected: 'අපි කරන්නෙමු'
--------------------
Incorrect: 'This is a test'
Corrected: 'This is a test'
Expected: 'This is a test'
--------------------
Incorrect: ''
Corrected: ''
Expected: ''
--------------------
Incorrect: 'මම'

In [17]:
formal_sinhala_conjugations = {
    "present": {
        "singular": {
            "1st person": {"මම": ["න්නෙමි", "මි"]},
            "2nd person": {"ඔබ": ["න්නෙහි", "හි"]},
            "3rd person masculine": {"ඔහු": ["යි"]},
            "3rd person feminine": {"ඇය": ["යි"]},
            "3rd person neuter": {"ඒ/එය": ["යි"]},
        },
        "plural": {
            "1st person": {"අපි": ["න්නෙමු", "මු"]},
            "2nd person": {"ඔබලා/නුඹලා": ["න්නෙහු", "හු"]},
            "3rd person": {"ඔවුන්": ["ති"]},
            "3rd person neuter": {"ඒවා": ["ති"]},
        },
    },
    "past": {
        "singular": {
            "1st person": {"මම": ["-ුවෙමි", "-ුයෙමි", "-ඇමි", "-අස්මි"]},
            "2nd person": {"ඔබ": ["-ුවෙහි", "-ුයෙහි", "-ඇයෙහි", "-අසෙහි"]},
            "3rd person masculine": {"ඔහු": ["-ුවේය", "-ුවා"]},
            "3rd person feminine": {"ඇය": ["-ුවාය", "-ුයාය"]},
            "3rd person neuter": {"ඒ/එය": ["-ුවේය", "-ුවා"]},
        },
        "plural": {
            "1st person": {"අපි": ["-ුවෙමු", "-ුයෙමු"]},
            "2nd person": {"ඔබලා/නුඹලා": ["-ුවෙහු", "-ුයෙහු"]},
            "3rd person": {"ඔවුන්": ["-ුවෝය", "-ුයෝය"]},
            "3rd person neuter": {"ඒවා": ["-ුවෝය", "-ුයෝය"]},
        },
    },
    "future": {
        "singular": {
            "1st person": {"මම": ["න්නම්"]},
            "2nd person": {"ඔබ": ["න්නෙහි"]},
            "3rd person masculine": {"ඔහු": ["යි", "න්නේය"]},
            "3rd person feminine": {"ඇය": ["යි", "න්නේය"]},
            "3rd person neuter": {"ඒ/එය": ["යි", "න්නේය"]},
        },
        "plural": {
            "1st person": {"අපි": ["න්නෙමු", "මු"]},
            "2nd person": {"ඔබලා/නුඹලා": ["න්නෙහු"]},
            "3rd person": {"ඔවුන්": ["ති", "න්නාහ"]},
            "3rd person neuter": {"ඒවා": ["ති", "න්නාහ"]},
        },
    },
}

def extract_verb_stem(verb):
    suffixes = [
        "නවා", "න", "ඉම", "න්න", "යි", "ති", "මි", "හි", "ව",
        "න්නෙමි", "න්නෙහි", "න්නෙමු", "න්නෙහු", "න්නම්", "න්නේය", "න්නාහ"
    ]
    for suffix in sorted(suffixes, key=len, reverse=True):
        if verb.endswith(suffix):
            return verb[:-len(suffix)]
    return verb

def detect_tense(verb):
    if any(verb.endswith(suffix) for suffix in ["මි", "නවා", "ති", "යි"]):
        return "present"
    elif any(verb.endswith(suffix) for suffix in ["-ුව", "-ුවෙ", "-ුය"]):
        return "past"
    elif any(verb.endswith(suffix) for suffix in ["න්නම්", "න්නේය", "න්නාහ"]):
        return "future"
    return "present"

def correct_sentence(sentence):
    words = sentence.split()
    if not words:
        return ""

    subject = words[0]
    verb_index = -1

    for i, word in enumerate(words[1:], start=1):
        if any(word.endswith(suffix) for suffix in formal_sinhala_conjugations["present"]["singular"]["1st person"]["මම"]):
            verb_index = i
            break

    if verb_index == -1:
        return sentence

    verb = words[verb_index]
    verb_stem = extract_verb_stem(verb)
    tense = detect_tense(verb)

    try:
        formality = "singular" if subject in ["මම", "ඔබ", "ඔහු", "ඇය", "ඒ/එය"] else "plural"
        person = (
            "1st person" if subject in ["මම", "අපි"] else
            "2nd person" if subject in ["ඔබ", "ඔබලා/නුඹලා"] else
            "3rd person"
        )

        endings = formal_sinhala_conjugations[tense][formality][person][subject]
        corrected_verb = verb_stem + endings[0]
        words[verb_index] = corrected_verb
        return " ".join(words)

    except KeyError:
        return sentence

# Test Cases
test_cases = [
    ("මම යනවා", "මම යන්නෙමි"),
    ("ඔබ යන්නෙවා", "ඔබ යන්නෙහි"),
    ("ඔහු යනවා", "ඔහු යයි"),
    ("අපි කරම", "අපි කරන්නෙමු"),
    ("ඔවුන් කරති", "ඔවුන් කරති"),
    ("මම කෑවා", "මම කෑවා"),
    ("ඔබ කෑවා", "ඔබ කෑවා"),
]

for incorrect_sentence, correct_sentence_expected in test_cases:
    corrected_sentence = correct_sentence(incorrect_sentence)
    print(f"Incorrect: '{incorrect_sentence}'")
    print(f"Corrected: '{corrected_sentence}'")
    print(f"Expected: '{correct_sentence_expected}'")
    print("-" * 20)


Incorrect: 'මම යනවා'
Corrected: 'මම යනවා'
Expected: 'මම යන්නෙමි'
--------------------
Incorrect: 'ඔබ යන්නෙවා'
Corrected: 'ඔබ යන්නෙවා'
Expected: 'ඔබ යන්නෙහි'
--------------------
Incorrect: 'ඔහු යනවා'
Corrected: 'ඔහු යනවා'
Expected: 'ඔහු යයි'
--------------------
Incorrect: 'අපි කරම'
Corrected: 'අපි කරම'
Expected: 'අපි කරන්නෙමු'
--------------------
Incorrect: 'ඔවුන් කරති'
Corrected: 'ඔවුන් කරති'
Expected: 'ඔවුන් කරති'
--------------------
Incorrect: 'මම කෑවා'
Corrected: 'මම කෑවා'
Expected: 'මම කෑවා'
--------------------
Incorrect: 'ඔබ කෑවා'
Corrected: 'ඔබ කෑවා'
Expected: 'ඔබ කෑවා'
--------------------


In [20]:
formal_sinhala_conjugations = {
    "present": {
        "singular": {
            "1st person": {"මම": ["න්නෙමි", "මි"]},
            "2nd person": {"ඔබ": ["න්නෙහි", "හි"]},
            "3rd person masculine": {"ඔහු": ["යි"]},
            "3rd person feminine": {"ඇය": ["යි"]},
            "3rd person neuter": {"ඒ/එය": ["යි"]},
        },
        "plural": {
            "1st person": {"අපි": ["න්නෙමු", "මු"]},
            "2nd person": {"ඔබලා/නුඹලා": ["න්නෙහු", "හු"]},
            "3rd person": {"ඔවුන්": ["ති"]},
            "3rd person neuter": {"ඒවා": ["ති"]},
        }
    },
    "past": {
        "singular": {
            "1st person": {"මම": ["ීමි", "ුවෙමි", "ුයෙමි", "ඇමි", "අස්මි"]},
            "2nd person": {"ඔබ": ["ුවෙහි", "ුව", "ුයෙහි", "ඇයෙහි", "අසෙහි"]},
            "3rd person masculine": {"ඔහු": ["ුවේය", "ුවා", "ුයේය", "ඇයේය", "ඇස්සය"]},
            "3rd person feminine": {"ඇය": ["ුවාය", "ුවා", "ුයාය", "ඇයාය", "ඇස්සාය"]},
            "3rd person neuter": {"ඒ/එය": ["ුවේය", "ුවා", "ුයේය", "ඇයේය", "ඇස්සය"]},
        },
        "plural": {
            "1st person": {"අපි": ["ුවෙමු", "ුව", "ුයෙමු", "ඇවමු", "ඇස්සුමු"]},
            "2nd person": {"ඔබලා/නුඹලා": ["ුවෙහු", "ුව", "ුයෙහු", "ඇවහු", "ඇස්සුහු"]},
            "3rd person": {"ඔවුන්": ["ුවෝය", "ුවා", "ුයෝය", "ඇවෝය", "ඇස්සෝය"]},
            "3rd person neuter": {"ඒවා": ["ුවෝය", "ුවා", "ුයෝය", "ඇවෝය", "ඇස්සෝය"]},
        }
    },
    "future": {
        "singular": {
            "1st person": {"මම": ["න්නම්", "මි"]},
            "2nd person": {"ඔබ": ["න්නෙහි", "හි"]},
            "3rd person masculine": {"ඔහු": ["යි", "න්නේය"]},
            "3rd person feminine": {"ඇය": ["යි", "න්නේය"]},
            "3rd person neuter": {"ඒ/එය": ["යි", "න්නේය"]},
        },
        "plural": {
            "1st person": {"අපි": ["න්නෙමු", "මු", "න්නෙම්"]},
            "2nd person": {"ඔබලා/නුඹලා": ["න්නෙහු", "හු"]},
            "3rd person": {"ඔවුන්": ["ති", "න්නාහ"]},
            "3rd person neuter": {"ඒවා": ["ති", "න්නාහ"]},
        }
    }
}

def extract_verb_stem(verb):
    suffixes = ["නවා", "න", "ඉම", "න්න", "න්නට", "න්නෙ", "න්නෙමි", "න්නෙහි", "න්නෙමු", "න්නෙහු", "ති", "යි", "වා", "න්නම්", "න්නේය", "න්නාහ", "ීමි", "ුවෙමි", "ුයෙමි", "ඇමි", "අස්මි", "ුවෙහි", "ුව", "ුයෙහි", "ඇයෙහි", "අසෙහි", "ුවේය", "ුවා", "ුයේය", "ඇයේය", "ඇස්සය", "ුවාය", "ුවා", "ුයාය", "ඇයාය", "ඇස්සාය", "ුවෙමු", "ුව", "ුයෙමු", "ඇවමු", "ඇස්සුමු", "ුවෙහු", "ුව", "ුයෙහු", "ඇවහු", "ඇස්සුහු", "ුවෝය", "ුවා", "ුයෝය", "ඇවෝය", "ඇස්සෝය","න්නෙම්","න්නෙ","ඉන්න"] # added "ඉන්න"
    for suffix in suffixes:
        if verb.endswith(suffix):
            return verb[:-len(suffix)]
    return verb

def correct_sentence(sentence):
    words = sentence.split()
    if not words:
        return ""

    subject = words[0]
    verb_index = -1

    for i, word in enumerate(words[1:]):
        if any(word.endswith(suffix) for suffix in ["වා", "නවා", "යි", "ති", "න්නම්", "න්නේය", "න්නාහ", "ීමි", "ුවෙමි", "ුයෙමි", "ඇමි", "අස්මි", "ුවෙහි", "ුව", "ුයෙහි", "ඇයෙහි", "අසෙහි", "ුවේය", "ුවා", "ුයේය", "ඇයේය", "ඇස්සය", "ුවාය", "ුවා", "ුයාය", "ඇයාය", "ඇස්සාය", "ුවෙමු", "ුව", "ුයෙමු", "ඇවමු", "ඇස්සුමු", "ුවෙහු", "ුව", "ුයෙහු", "ඇවහු", "ඇස්සුහු", "ුවෝය", "ුවා", "ුයෝය", "ඇවෝය", "ඇස්සෝය"]):
            verb_index = i + 1
            break

    if verb_index == -1:
        return sentence

    verb = words[verb_index]
    verb_stem = extract_verb_stem(verb)
    tense = "present"
    if any(verb.endswith(suffix) for suffix in ["ීමි", "ුවෙමි", "ුයෙමි", "ඇමි", "අස්මි", "ුවෙහි", "ුව", "ුයෙහි", "ඇයෙහි", "අසෙහි", "ුවේය", "ුවා", "ුයේය", "ඇයේය", "ඇස්සය", "ුවාය", "ුවා", "ුයාය", "ඇයාය", "ඇස්සාය", "ුවෙමු", "ුව", "ුයෙමු", "ඇවමු", "ඇස්සුමු", "ුවෙහු", "ුව", "ුයෙහු", "ඇවහු", "ඇස්සුහු", "ුවෝය", "ුවා", "ුයෝය", "ඇවෝය", "ඇස්සෝය"]):
        tense = "past"
    elif any(verb.endswith(suffix) for suffix in ["න්නම්","න්නේය","න්නාහ"]):
        tense = "future"

    try:
        endings = formal_sinhala_conjugations[tense]["singular" if subject in ["මම", "ඔබ", "ඔහු", "ඇය", "ඒ/එය"] else "plural"][
            "1st person" if subject in ["මම", "අපි"] else
            "2nd person" if subject in ["ඔබ", "ඔබලා/නුඹලා"] else
            "3rd person"
        ][subject]

        best_ending = None
        for ending in endings:
            if verb.endswith(ending) or not any(verb.endswith(suffix) for suffix in ["වා", "නවා", "යි", "ති", "න්නම්", "න්නේය", "න්නාහ", "ීමි", "ුවෙමි", "ුයෙමි", "ඇමි", "අස්මි", "ුවෙහි", "ුව", "ුයෙහි", "ඇයෙහි", "අසෙහි", "ුවේය", "ුවා", "ුයේය", "ඇයේය", "ඇස්සය", "ුවාය", "ුවා", "ුයාය", "ඇයාය", "ඇස්සාය", "ුවෙමු", "ුව", "ුයෙමු", "ඇවමු", "ඇස්සුමු", "ුවෙහු", "ුව", "ුයෙහු", "ඇවහු", "ඇස්සුහු", "ුවෝය", "ුවා", "ුයෝය", "ඇවෝය", "ඇස්සෝය"]):
                best_ending = ending
                break
        if best_ending is None:
            best_ending = endings[0]

        corrected_verb = verb_stem + best_ending
        words[verb_index] = corrected_verb
        return " ".join(words)

    except KeyError:
        return sentence

test_cases = [
    ("මම යනවා", "මම යන්නෙමි"),
    ("ඔයා යනවා", "ඔයා යන්නෙහි"),  # Corrected expected output
    ("මම කනවා", "මම කන්නෙමි"),
    ("ඔයා කන්නවා", "ඔයා කන්නෙහි"),  # Corrected expected output
    ("මම ගමන් කරනවා", "මම ගමන් කරන්නෙමි"),
    ("ඔහු කරයි", "ඔහු කරයි"),
    ("ඔවුන් කරති", "ඔවුන් කරති"),
    ("මම යමි", "මම යන්නෙමි"),
    ("අපි කරමු", "අපි කරන්නෙමු"),
    ("This is a test", "This is a test"),
    ("", ""),
    ("මම", "මම"),
    ("ඔබ කනවා", "ඔබ කන්නෙහි"),
    ("මම කෙරුවා", "මම කෙරුවා"), # Past tense (not handled yet)
    ("ඔබ කෙරුවා", "ඔබ කෙරුවා"), # Past tense (not handled yet)
    ("මම කරමි","මම කරන්නෙමි"),
    ("මම කරන්නම්","මම කරන්නම්"), #future tense
    ("ඔබ කරන්නෙහි","ඔබ කරන්නෙහි"),#future tense
    ("ඔහු කරන්නේය","ඔහු කරන්නේය"),#future tense
    ("අපි කරන්නෙම","අපි කරන්නෙමු"),#future tense
    ("ඔවුන් කරන්නාහ","ඔවුන් කරන්නාහ"),#future tense
    ("මම කෑවා","මම කෑවා"),#past tense
    ("ඔබ කෑවා","ඔබ කෑවා"),#past tense
    ("ඔහු කෑවා","ඔහු කෑවා"),#past tense
    ("අපි කෑවා","අපි කෑවා"),#past tense
    ("ඔවුන් කෑවා","ඔවුන් කෑවා"),#past tense
    ("මම ගියා","මම ගියා"),#past tense
    ("ඔබ ගියා","ඔබ ගියා"),#past tense
    ("ඔහු ගියා","ඔහු ගියා"),#past tense
    ("අපි ගියා","අපි ගියා"),#past tense
    ("ඔවුන් ගියා","ඔවුන් ගියා"),#past tense
    ("මම ඉන්නවා","මම ඉන්නෙමි"),
    ("ඔබ ඉන්නවා","ඔබ ඉන්නෙහි"),
    ("ඔහු ඉන්නවා","ඔහු ඉන්නවා"),
    ("අපි ඉන්නවා","අපි ඉන්නෙමු"),
    ("ඔවුන් ඉන්නවා","ඔවුන් ඉන්නවා")
]

for incorrect_sentence, correct_sentence_expected in test_cases:
    corrected_sentence = correct_sentence(incorrect_sentence)
    print(f"Incorrect: '{incorrect_sentence}'")
    print(f"Corrected: '{corrected_sentence}'")
    print(f"Expected: '{correct_sentence_expected}'")
    print("-" * 20)


Incorrect: 'මම යනවා'
Corrected: 'මම යන්නෙමි'
Expected: 'මම යන්නෙමි'
--------------------
Incorrect: 'ඔයා යනවා'
Corrected: 'ඔයා යනවා'
Expected: 'ඔයා යන්නෙහි'
--------------------
Incorrect: 'මම කනවා'
Corrected: 'මම කන්නෙමි'
Expected: 'මම කන්නෙමි'
--------------------
Incorrect: 'ඔයා කන්නවා'
Corrected: 'ඔයා කන්නවා'
Expected: 'ඔයා කන්නෙහි'
--------------------
Incorrect: 'මම ගමන් කරනවා'
Corrected: 'මම ගමන් කරන්නෙමි'
Expected: 'මම ගමන් කරන්නෙමි'
--------------------
Incorrect: 'ඔහු කරයි'
Corrected: 'ඔහු කරයි'
Expected: 'ඔහු කරයි'
--------------------
Incorrect: 'ඔවුන් කරති'
Corrected: 'ඔවුන් කරති'
Expected: 'ඔවුන් කරති'
--------------------
Incorrect: 'මම යමි'
Corrected: 'මම යමි'
Expected: 'මම යන්නෙමි'
--------------------
Incorrect: 'අපි කරමු'
Corrected: 'අපි කරමු'
Expected: 'අපි කරන්නෙමු'
--------------------
Incorrect: 'This is a test'
Corrected: 'This is a test'
Expected: 'This is a test'
--------------------
Incorrect: ''
Corrected: ''
Expected: ''
--------------------
Incorrect: 'මම'